**Optimal Replacement Policy (Belady Algorithm)** for Cache Management Policy

**Terminology**

**block_trace** - list of sequestial block request from CPU to memory/disk

**cold_miss** - compulsory misses when the caceh is empty and it is the first time a block has been requested

**cap_miss** - capacity miss when cache run out of space and had to **evict** a block from cache to bring the requested one into cache

**hit** - if block found in cache

**miss** - if block is not found in cache, also known as **fault**

We want to optimize (minimize) the number of faults (or maximize number of hits)

In [ ]:
import numpy as np
import random
import pandas as pd
from tqdm import tqdm

In [ ]:
#length of block trace
l = 20

#lowest value of block number 
b_low = 1

#highest value of block number
b_high = 7

block_trace = np.random.randint(b_low, b_high, size=(l)).tolist()

In [ ]:
data = pd.read_csv('cheetah.cs.fiu.edu-110108-113008.1.blkparse', header=None)

In [ ]:
data

In [ ]:
data = data[3:]

In [ ]:
df = pd.DataFrame(data[0].str.split(' ',9).tolist())

In [ ]:
blocktrace = df[3].tolist()
blocktrace = [int(x) for x in blocktrace]

In [ ]:
blocktrace

In [ ]:
len(blocktrace)

In [ ]:
len(set(blocktrace))

In [ ]:
#sample block trace we use for testing and consistency 
block_trace = [0,1,2,0,1,3,0,3,1,2,1]

In [ ]:
def checkindex(item, ilist):
  
  try:
    return ilist.index(item)
  except ValueError:
    return -1 

In [ ]:
def optimal(blocktrace, frame):
    cache = []

    hit, miss = 0, 0

    print("block", "\t\t hit", "\t\t miss", "\t\t cache")
    for i, block in enumerate(blocktrace):
        if block in cache:
            hit += 1
            pass

        elif len(cache) < frame:
            cache.append(block)
            miss += 1

        else:
            remain_block_trace = blocktrace[i:]

            indexed = [checkindex(j, remain_block_trace) for j in cache]

            indexed2 = list(filter(lambda a: a != -1, indexed))
      
            if len(indexed2) == 1:
                cache.pop()
                if remain_block_trace[indexed2[0]] not in cache:
                    cache.pop()
                    cache.append(remain_block_trace[indexed2[0]])
                    cache.append(block)
                    miss += 1

                elif len(indexed2) == 0:
                    cache.pop(0)
                    cache.append(block)
                    miss += 1

                else:
                    cache.remove(remain_block_trace[max(indexed2)])
                    cache.append(block)
                    miss += 1


        print(block, "\t\t", hit, "\t\t", miss, "\t\t", cache)

    hitrate = hit / (hit + miss)
    print("\n\nhitrate: ", hitrate)

    return hitrate


In [ ]:
def FIFO(blocktrace, frame):
    '''
      FIFO - First In First Out
    '''
  
    cache = []

    hit, miss = 0, 0

    print("block", "\t\t hit", "\t\t miss", "\t\t cache")
    for block in blocktrace:
        if block in cache:
            hit += 1
            pass

        elif len(cache) < frame:
            cache.append(block)
            miss += 1

        else:
            cache.pop(0)
            cache.append(block)
            miss += 1

        print(block, "\t\t", hit, "\t\t", miss, "\t\t", cache)

    hitrate = hit / (hit + miss)
    print("\n\nhitrate: ", hitrate)
  
    return hitrate

In [ ]:
block_trace

In [ ]:
def FIFO_np(blocktrace, frame):
    
    cache = np.zeros((frame), dtype=int)
    hit, miss = 0, 0
    
    #print("block", "\t\t hit", "\t\t miss", "\t\t cache", \
     #     "\t\tpointer")

    pointer = 0
    for block in tqdm(blocktrace):
        if block in cache:
            hit += 1
        
        elif frame - np.count_nonzero(cache) != 0:
            cache[np.where(cache==0)[0][0]] = block
            miss += 1
        
        else:
            cache[pointer % frame] = block
            pointer += 1
            miss += 1
        
      #  print(block, "\t\t", hit, "\t\t", miss, "\t\t", cache,\
       #       "\t\t", pointer)
    
    hitrate = hit / (hit + miss)
    print(hitrate)
    
    return hitrate
            

In [ ]:
block_trace

In [ ]:
FIFO(block_trace, 3)

In [ ]:
FIFO_np(block_trace, 3)

In [ ]:
hitrateFIFO = FIFO_np(blocktrace, 35000)

In [ ]:
def random_replacement(blocktrace, frame):
  
    cache = []
    hit, miss = 0, 0

    print("block", "\t\t hit", "\t\t miss", "\t\t cache")
    for i, block in enumerate(blocktrace):
        if block in cache:
            hit += 1
            pass

        elif len(cache) < frame:
            cache.append(block)
            miss += 1

        else:
            random.shuffle(cache)
            cache.pop()
            cache.append(i)
            miss += 1


    print(block, "\t\t", hit, "\t\t", miss, "\t\t", cache)
   
    hitrate = hit / (hit + miss)
    print("\n\nhitrate: ", hitrate)
    
    return hitrate

In [ ]:
random_replacement(block_trace, 3)

In [ ]:
def LFU(blocktrace, frame):
  
  cache = []
  hit, miss = 0, 0

  print("block", "\t\t hit", "\t\t miss", "\t\t cache")
  for i, block in enumerate(blocktrace):
    if block in cache:
      hit += 1
      pass

    elif len(cache) < frame:
      cache.append(block)
      miss += 1
      
    else:
      
      past_blocktrace = blocktrace[:i]
      cache_element_frequency = list(map(lambda x: past_blocktrace.count(x), cache))
      cache.pop(cache_element_frequency.index(min(cache_element_frequency)))
      cache.append(block)
      miss += 1
    
    print(block, "\t\t", hit, "\t\t", miss, "\t\t", cache)
   
  hitrate = hit / (hit + miss)
  print("\n\nhitrate: ", hitrate)

In [ ]:
LFU(block_trace, 3)

In [ ]:
def LRU(blocktrace, frame):
  cache = []
  hit, miss = 0, 0

  print("block", "\t\t hit", "\t\t miss", "\t\t cache")
  for i, block in enumerate(blocktrace):
    if block in cache:
      hit += 1
      pass

    elif len(cache) < frame:
      cache.append(block)
      miss += 1
      
    else:
      past_blocktrace = blocktrace[:i]
      past_blocktrace.reverse()
      
      indexed = [checkindex(j, past_blocktrace) for j in cache]

      #print('index: ', indexed)
      cache.pop(indexed.index(max(indexed)))
      cache.append(block)
      miss += 1
    
    print(block, "\t\t", hit, "\t\t", miss, "\t\t", cache)
   
  hitrate = hit / (hit + miss)
  print("\n\nhitrate: ", hitrate)

In [ ]:
LRU(block_trace, 3)

In [ ]:
l = np.zeros((14), dtype=int)

In [ ]:
l[3] = 2

In [ ]:
len(l) - np.count_nonzero(l)

l.shape

In [ ]:
np.where(l == 0)[0][1]

In [ ]:
np.where(l == 0)

In [ ]:
l= np.array([1,2,3,0,6,7,0])

In [ ]:
l

In [ ]:
np.where(l==0)[0].shape[0]

In [ ]:
l2 = [1,2,3,4,5,6,9,9]
l2.index(3)

In [ ]:
blocktrace.index(283193152)

In [ ]:
if 3 in l:
    print('yes')

In [ ]:
l[1]

In [ ]:
X[1].toarray()